In [ ]:
import numpy as np
import random
random.seed(1)
np.random.seed(1)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import time
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import VotingClassifier
import random
from sklearn import svm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fulldata.csv",index_col=0) 

In [ ]:
x = data.drop(['label'], axis = 1)
y = data['label']

In [ ]:
from collections import Counter
Counter(y),x.shape

In [ ]:
!pip install -U imbalanced-learn
!pip install -U scikit-learn


In [ ]:
from sklearn import preprocessing
scaler = StandardScaler()
X= pd.DataFrame(scaler.fit_transform(x))
X.columns=x.columns
le = preprocessing.LabelEncoder()
le.fit(y)
Y=le.transform(y)

In [ ]:
import numpy as np
random.seed(1)
np.random.seed(1)
X_train, X_test, y_train, y_test=  train_test_split(X,Y,stratify=Y,test_size=0.2)
X_train.shape,X_test.shape

In [ ]:
import sklearn
sklearn.__version__


In [ ]:
random.seed(1)
np.random.seed(1)
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
# fit model and predict clusters
dta1_x=X_train[(y_train == 1)]
dta0_x=X_train[(y_train == 0)]
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
silhouette_avg = []
for num_clusters in range_n_clusters: 
  # initialise kmeans
  kmeans = KMeans(n_clusters=num_clusters)
  kmeans.fit(dta1_x)
  cluster_labels = kmeans.labels_

  # silhouette score
  silhouette_avg.append(silhouette_score(dta1_x, cluster_labels))
plt.plot(range_n_clusters,silhouette_avg,'bx-')
for i, txt in enumerate(range_n_clusters):
    plt.annotate('$k={}$'.format(txt),(range_n_clusters[i], silhouette_avg[i]))
plt.xlabel('Values of K') 
plt.ylabel('Silhouette score') 
#plt.title('Silhouette analysis For Optimal k')
from google.colab import files
plt.savefig("fig2.png")
files.download("fig2.png")

In [ ]:
#Proposed method
random.seed(1)
np.random.seed(1)
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.feature_selection import f_classif,SelectFpr
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.calibration import CalibratedClassifierCV
from numpy import sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import *
from numpy import mean
from numpy import std

rus= RandomUnderSampler(random_state=1)
scaler = StandardScaler()
le = preprocessing.LabelEncoder()

model = KMeans(n_clusters=2)
# fit model and predict clusters
dta1_x=X_train[(y_train == 1)]
dta0_x=X_train[(y_train == 0)]
yhat = model.fit_predict(dta1_x)
# retrieve unique clusters
clusters = unique(yhat)
twod_list1 = [] 
for i in unique(yhat):
  clust_x=dta1_x.iloc[where(yhat == i)]
  dfX=pd.concat([clust_x,dta0_x], axis=0).reset_index(drop=True)
  #Creating balanced dataset
  dfY=pd.DataFrame(np.append(np.ones(clust_x.shape[0]),np.zeros(dta0_x.shape[0])))
  #Balancing dataset using undersampling
  dfXX, dfYY = rus.fit_resample(dfX, dfY[0].values)
  DFXX= pd.DataFrame(scaler.fit_transform(dfXX))
  DFXX.columns=dfXX.columns
  le.fit(dfYY)
  DFYY=le.transform(dfYY) 
  #Feature selection from balanced data
  selector = SelectFpr(alpha=0.05)
  selector.fit(DFXX,DFYY)
  feature_imp=DFXX.columns[selector.get_support(indices=False)]
  twod_list1.append(feature_imp)  

flat_list = [item for sublist in twod_list1 for item in sublist]
# getting the elements frequencies using Counter class
elements_count = Counter(np.array(flat_list))
# Union of features
inter={k for (k,v) in elements_count.items() if v > 0}

X_train111=X_train[inter]
y_train1=y_train
X_test1=X_test[inter]

#Classification

clf1 = LogisticRegression(class_weight='balanced').fit(X_train111, y_train1)
clf11 = LogisticRegression().fit(X_train111, y_train1)
clf2 = RandomForestClassifier(class_weight='balanced').fit(X_train111, y_train1)
clf3 = svm.SVC(class_weight='balanced',probability=True).fit(X_train111, y_train1)
clf4 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(class_weight='balanced'), cv=5).fit(X_train111, y_train1)
clf44 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(), cv=5).fit(X_train111, y_train1)
clf5 = VotingClassifier(estimators=[('lr', clf11), ('gnb', clf44)],voting='soft').fit(X_train111, y_train1)
for clf, label in zip([clf1, clf2, clf3, clf4,clf5], ['Logistic Regression', 'Random Forest', 'SVMR','SVML','Ensemble']):
  # define the evaluation method
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  # evaluate the model on the dataset
  n_scores = cross_val_score(clf, X_train111, y_train1, scoring='roc_auc', cv=cv, n_jobs=-1)
  # report performance
  print('Mean AUC: %.3f (%.3f) [%s]' % (mean(n_scores), std(n_scores), label))
  y_pred=clf.predict(X_test1)
  print("Accuracy",accuracy_score(y_test, y_pred),label)
  print("Precision", precision_score(y_test, y_pred),label)
  sensitivity=recall_score(y_test, y_pred)
  print("recall/sensitivity",sensitivity,label)
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  specificity = tn / (tn+fp)
  print("GMean",sqrt(sensitivity*specificity),label)
  print("F1 score",f1_score(y_test, y_pred),label)
  print("AUC:",metrics.roc_auc_score(y_test, clf.predict_proba(X_test1)[:,1]),label)
  print("################################################")

In [ ]:
#without clustering only feature selection from imbalanced data
random.seed(1)
np.random.seed(1)
selector = SelectFpr(alpha=0.05)
X_train11=selector.fit_transform(X_train,y_train)
y_train1=y_train
X_test11=selector.transform(X_test)
X_train11.shape,y_train1.shape,X_test11.shape,y_test.shape


In [ ]:
#From balanced data without clustering
random.seed(1)
np.random.seed(1)
from sklearn.feature_selection import f_classif,SelectFpr
from imblearn.over_sampling import SMOTE, ADASYN,RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler,TomekLinks,NearMiss
import collections
rus= SMOTE(random_state=1)
#rus=TomekLinks()
scaler = StandardScaler()
le = preprocessing.LabelEncoder()

dfXX, dfYY = rus.fit_resample(X_train, y_train)
DFXX= pd.DataFrame(scaler.fit_transform(dfXX))
DFXX.columns=dfXX.columns
le.fit(dfYY)
DFYY=le.transform(dfYY) 

selector = SelectFpr(alpha=0.05)
selector.fit(DFXX,DFYY)
feature_imp=DFXX.columns[selector.get_support(indices=False)]
X_train11=DFXX[feature_imp]
y_train1=DFYY
X_test11=X_test[feature_imp]
X_train11.shape,y_train1.shape,X_test11.shape,y_test.shape


In [ ]:
random.seed(1)
np.random.seed(1)
from sklearn.calibration import CalibratedClassifierCV
from numpy import sqrt
X_train111=X_train11
X_test1=X_test11
clf1 = LogisticRegression(class_weight='balanced').fit(X_train111, y_train1)
clf11 = LogisticRegression().fit(X_train111, y_train1)
clf2 = RandomForestClassifier(class_weight='balanced').fit(X_train111, y_train1)
clf3 = svm.SVC(class_weight='balanced',probability=True).fit(X_train111, y_train1)
clf4 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(class_weight='balanced'), cv=5).fit(X_train111, y_train1)
clf44 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(), cv=5).fit(X_train111, y_train1)
clf5 = VotingClassifier(estimators=[('lr', clf11), ('gnb', clf44)],voting='soft').fit(X_train111, y_train1)
for clf, label in zip([clf1, clf2, clf3, clf4,clf5], ['Logistic Regression', 'Random Forest', 'SVMR','SVML','Ensemble']):
  # define the evaluation method
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  # evaluate the model on the dataset
  n_scores = cross_val_score(clf, X_train111, y_train1, scoring='roc_auc', cv=cv, n_jobs=-1)
  # report performance
  print('Mean AUC: %.3f (%.3f) [%s]' % (mean(n_scores), std(n_scores), label))
  y_pred=clf.predict(X_test1)
  #plot_confusion_matrix(clf,X_test3, y_test)
  print("Accuracy",accuracy_score(y_test, y_pred),label)
  print("Precision", precision_score(y_test, y_pred),label)
  sensitivity=recall_score(y_test, y_pred)
  print("recall/sensitivity",sensitivity,label)
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  specificity = tn / (tn+fp)
  print("GMean",sqrt(sensitivity*specificity),label)
  print("F1 score",f1_score(y_test, y_pred),label)
  print("AUC:",metrics.roc_auc_score(y_test, clf.predict_proba(X_test1)[:,1]),label)
  print("################################################")

In [ ]:
random.seed(1)
np.random.seed(1)
from sklearn.calibration import CalibratedClassifierCV
from numpy import sqrt
X_train111=X_train11
X_test1=X_test11
clf1 = LogisticRegression().fit(X_train111, y_train1)
clf11 = LogisticRegression().fit(X_train111, y_train1)
clf2 = RandomForestClassifier().fit(X_train111, y_train1)
clf3 = svm.SVC(probability=True).fit(X_train111, y_train1)
clf4 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(), cv=5).fit(X_train111, y_train1)
clf44 = CalibratedClassifierCV(base_estimator=svm.LinearSVC(), cv=5).fit(X_train111, y_train1)
clf5 = VotingClassifier(estimators=[('lr', clf11), ('gnb', clf44)],voting='soft').fit(X_train111, y_train1)
for clf, label in zip([clf1, clf2, clf3, clf4,clf5], ['Logistic Regression', 'Random Forest', 'SVMR','SVML','Ensemble']):
  # define the evaluation method
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  # evaluate the model on the dataset
  n_scores = cross_val_score(clf, X_train111, y_train1, scoring='roc_auc', cv=cv, n_jobs=-1)
  # report performance
  print('Mean AUC: %.3f (%.3f) [%s]' % (mean(n_scores), std(n_scores), label))
  y_pred=clf.predict(X_test1)
  print("Accuracy",accuracy_score(y_test, y_pred),label)
  print("Precision", precision_score(y_test, y_pred),label)
  sensitivity=recall_score(y_test, y_pred)
  print("recall/sensitivity",sensitivity,label)
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  specificity = tn / (tn+fp)
  print("GMean",sqrt(sensitivity*specificity),label)
  print("F1 score",f1_score(y_test, y_pred),label)
  print("AUC:",metrics.roc_auc_score(y_test, clf.predict_proba(X_test1)[:,1]),label)
  print("################################################")